In [35]:
from google.cloud import bigquery
import pandas as pd
import numpy as np

from google.oauth2 import service_account

key_path = "crunchbase-startups-b791b011b22c.json"

credentials = service_account.Credentials.from_service_account_file(
key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

sql = """
    SELECT *
    FROM `crunchbase-startups.startups.join_table`
"""

df = client.query(sql).to_dataframe()

In [36]:
category_encoded1 = np.zeros(len(df))
category_encoded2 = np.zeros(len(df))
for c in set(np.array(df["category_code"])):
  if c == None:
    continue
  stat_f1 = np.average(df["status_encoded_field1"][df["category_code"] == c])
  stat_f2 = np.average(df["status_encoded_field2"][df["category_code"] == c])
  category_encoded1[df["category_code"]==c], category_encoded2[df["category_code"]==c] = stat_f1, stat_f2
df["category_encoded1"], df["category_encoded2"] = category_encoded1, category_encoded2

In [42]:
country_encoded1 = np.zeros(len(df))
country_encoded2 = np.zeros(len(df))
for c in set(np.array(df["country_code"])):
  if c == None:
    continue
  stat_f1 = np.average(df["status_encoded_field1"][df["country_code"] == c])
  stat_f2 = np.average(df["status_encoded_field2"][df["country_code"] == c])
  country_encoded1[df["country_code"]==c], country_encoded2[df["country_code"]==c] = stat_f1, stat_f2
df["country_encoded1"], df["country_encoded2"] = country_encoded1, country_encoded2

In [46]:
state_encoded1 = np.zeros(len(df))
state_encoded2 = np.zeros(len(df))
for c in set(np.array(df["state_code"])):
  if c == None:
    continue
  stat_f1 = np.average(df["status_encoded_field1"][df["state_code"] == c])
  stat_f2 = np.average(df["status_encoded_field2"][df["state_code"] == c])
  state_encoded1[df["state_code"]==c], state_encoded2[df["state_code"]==c] = stat_f1, stat_f2
df["state_encoded1"], df["state_encoded2"] = state_encoded1, state_encoded2